# Interpolate Calanus
Interpolate (univariate) the observations of _Calanus finmarchicus_ and _Calanus helgolandicus_.

The fields are prepared over a region covering the Northeast Atlantic.      
Different resolutions are tested: 0.5° X 0.5°, 1° X 1° and 2° X 2°.

In order to modify the directories, the spatial resolution, edit the configuration file [config.jl](../src/config.jl).

In [1]:
using Dates
using NCDatasets
using PyPlot
using DIVAnd
using Statistics
const plt=PyPlot
using PyCall
using PyPlot
include("../src/InterpCalanus.jl")

Main.InterpCalanus

In [2]:
include("../src/config.jl")

[ Info: Workig at resoluton 2.0° X 2.0°
[ Info: Observation file already downloaded
[ Info: Bathymetry file already downloaded


"#456A30"

## Install `Cartopy`
Set `usecartopy` to `false` if you don't want to use it.

In [3]:
if usecartopy
    @info("Will try to use Cartopy for the plots")
    cartopy = PyCall.pyimport("cartopy")
    ccrs = PyCall.pyimport("cartopy.crs")
    cfeature = PyCall.pyimport("cartopy.feature")
    coast = cfeature.GSHHSFeature(scale="full");
    cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
    mainproj = ccrs.Mercator(central_longitude=0.5*(domain[1] + domain[2]), 
        min_latitude=domain[3], max_latitude=domain[4])
    datacrs = ccrs.PlateCarree();
else
    @info("Won't use cartopy")
end

[ Info: Won't use cartopy


## User inputs
### Type of analysis to be performed
Set the values (`true` or `false`) for the parameters `run_year` and `run_month = true in `config.jl`.

## Grid configuration

In [4]:
longrid = domain[1]:dlon:domain[2]
latgrid = domain[3]:dlat:domain[4]
@info("Size of the grid: $(length(longrid)) X $(length(latgrid))")

[ Info: Size of the grid: 17 X 13


### Compute metrics

In [5]:
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);
@info(size(xi));

[ Info: (17, 13)


### Prepare land-sea mask
Using depth ≃ 5 meters

In [6]:
bx, by, b = DIVAnd.load_bath(bathyfile, true, longrid, latgrid)
@show size(b);

_, _, mask = DIVAnd.load_mask(bathyfile, true, longrid, latgrid, 5.0);
# Remove Mediterranean Sea
maskindex = findall((xi .>= 0.) .& (yi .<= 47.))
mask[maskindex] .= 0.;

size(b) = (17, 13)


## Data reading
### Read the CSV files and separate by columns

In [7]:
@time lon, lat, dates, calanus_finmarchicus, calanus_helgolandicus = InterpCalanus.read_data_calanus(datafile);

  1.257120 seconds (5.63 M allocations: 189.201 MiB, 5.26% gc time, 67.06% compilation time)


[ Info: (-20.447, 11.703)
[ Info: (41.2567, 66.808)


## Analysis
### 1. Basic analysis: all data together
- The objective is to show the results of the analysis with short, medium and large correlation lengths.
- The background field is set to zero: no observations means that there is 'nothing'.

In [8]:
L = 2.5
epsilon2 = 5.

5.0

In [9]:
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (L, L), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (L, L), epsilon2);

 20.663722 seconds (47.96 M allocations: 2.512 GiB, 4.47% gc time, 98.53% compilation time)
  0.328528 seconds (2.07 M allocations: 137.971 MiB, 9.39% gc time, 11.13% compilation time)


### 2. Long correlation length

In [10]:
Llong = 100
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Llong, Llong), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Llong, Llong), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_large.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Llong), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  1.870886 seconds (6.33 M allocations: 349.095 MiB, 3.79% gc time, 85.74% compilation time)
  0.293155 seconds (2.05 M allocations: 136.975 MiB, 9.96% gc time)


### 3. Short correlation length

In [11]:
Lshort = 0.5
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Lshort, Lshort), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Lshort, Lshort), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_short.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Lshort), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  0.263590 seconds (2.05 M allocations: 136.439 MiB, 6.84% gc time)
  0.243638 seconds (2.05 M allocations: 136.439 MiB)


## Time analysis
### Yearly
In these yearly analysis all the data are taken, whatever the month of the observation.     

#### Create the (empty) netCDF files:

In [12]:
resfileyear1 = joinpath(resdir, "calanus_finmarchicus_year.nc")
resfileyear2 = joinpath(resdir, "calanus_helgolandicus_year.nc")

InterpCalanus.create_nc_results_time(resfileyear1, longrid, latgrid,  
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=domain,
                      aphiaID=Int32(104464), L, epsilon2);

InterpCalanus.create_nc_results_time(resfileyear2, longrid, latgrid,  
                      "Calanus helgolandicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus helgolandicus",
                      domain=domain,
                      aphiaID=Int32(104466), L, epsilon2);

#### Loop on the years

In [13]:
if run_year 
    for (iii, yyyy) in enumerate(yearmin:yearmax)
        goodyear = findall(Dates.year.(dates) .== yyyy)
        @info("Number of observations for year $(yyyy): $(length(goodyear))")

        # Analysis
        f_finmarchicus, s_fin = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear] .- 0. * mean(calanus_finmarchicus[goodyear]), (L, L), epsilon2);
        f_helgolandicus, s_hel = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_helgolandicus[goodyear] .- 0. * mean(calanus_helgolandicus[goodyear]), (L, L), epsilon2);
        
        # Error field
        cpme = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear], L, epsilon2);
        
        #cpme_helgo = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
        #    calanus_helgolandicus[goodyear], L, epsilon2);
        
        if makeplot
            # Figures
            figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), 
                "Calanus finmarchicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), 
                "Calanus helgolandicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "error_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_error(longrid, latgrid, cpme, lon[goodyear], lat[goodyear], 
                "Relative error field", figname, mainproj)
        end
        
        #figname = joinpath(figdir, "error_calanus_helgolandicus_$(yyyy).jpg")
        #PlottingCalanus.make_plot_error(longrid, latgrid, cpme_helgo, lon[goodyear], lat[goodyear], 
        #    "Relative error field", figname, mainproj)
        
        # write single netCDF files
        InterpCalanus.create_nc_results(joinpath(resdirnc, "calanus_finmarchicus_$(yyyy).nc"), 
                      longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                      f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), L, epsilon2,
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=domain,
                      aphiaID=Int32(104464)
                   )
        
        InterpCalanus.write_nc_error(joinpath(resdirnc, "calanus_finmarchicus_$(yyyy).nc"), cpme)

        InterpCalanus.create_nc_results(joinpath(resdirnc, "calanus_helgolandicus_$(yyyy).nc"), 
                  longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                  f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), L, epsilon2, 
                  "Calanus helgolandicus"; valex=-999.9, 
                  varname="abundance",
                  long_name="Abundance of Calanus helgolandicus",
                  domain=domain,
                  aphiaID=Int32(104466)
               )
        
        InterpCalanus.write_nc_error(joinpath(resdirnc, "calanus_helgolandicus_$(yyyy).nc"), cpme)
        
        # Write common netCDF files
        Dataset(resfileyear1, "a") do ds
            ds["time"][iii] = (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value
            ds["error"][:,:,iii] = cpme
            ds["abundance"][:,:,iii] = f_finmarchicus
        end

        Dataset(resfileyear2, "a") do ds
            ds["time"][iii] = (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value
            ds["error"][:,:,iii] = cpme
            ds["abundance"][:,:,iii] = f_helgolandicus
        end
        
        # geoTIFF files
        InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_finmarchicus_$(yyyy).tif"), f_finmarchicus, domain)
        InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_helgolandicus_$(yyyy).tif"), f_helgolandicus, domain)
        InterpCalanus.write_geotiff(joinpath(resdirtif, "calanus_finmarchicus_error_$(yyyy).tif"), cpme, domain)
        #InterpCalanus.write_geotiff(joinpath(resdir, "calanus_helgolandicus_error_$(yyyy).tif"), cpme_helgo, dom)

    end
end

[ Info: Number of observations for year 1959: 2364
[ Info: Number of observations for year 1960: 2321
[ Info: Number of observations for year 1961: 2382
[ Info: Number of observations for year 1962: 2564
[ Info: Number of observations for year 1963: 2465
[ Info: Number of observations for year 1964: 2556
[ Info: Number of observations for year 1965: 2777
[ Info: Number of observations for year 1966: 2422
[ Info: Number of observations for year 1967: 2679
[ Info: Number of observations for year 1968: 2489
[ Info: Number of observations for year 1969: 2863
[ Info: Number of observations for year 1970: 2701
[ Info: Number of observations for year 1971: 2964
[ Info: Number of observations for year 1972: 2808
[ Info: Number of observations for year 1973: 2788
[ Info: Number of observations for year 1974: 2600
[ Info: Number of observations for year 1975: 2490
[ Info: Number of observations for year 1976: 2361
[ Info: Number of observations for year 1977: 2267
[ Info: Number of observations 

### Monthly

In [14]:
resfilemonth1 = joinpath(resdir, "calanus_finmarchicus_year_month.nc")
resfilemonth2 = joinpath(resdir, "calanus_helgolandicus_year_month.nc")

InterpCalanus.create_nc_results_time(resfilemonth1, longrid, latgrid,  
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=domain,
                      aphiaID=Int32(104464), L, epsilon2);

InterpCalanus.create_nc_results_time(resfilemonth2, longrid, latgrid,  
                      "Calanus helgolandicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus helgolandicus",
                      domain=domain,
                      aphiaID=Int32(104466), L, epsilon2);

In [15]:
if run_month
    iii = 0
    for yyyy in 1959:2018

        for m in 1:12

            iii += 1

            mm = lpad(string(m), 2, '0')
            # Select the good observations
            goodtime = findall( (Dates.year.(dates) .== yyyy) .& (Dates.month.(dates) .== m) )
            @info("Found $(length(goodtime)) dates points for year $(yyyy) | month $(mm)")

            if length(goodtime) > 0

                f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                    calanus_finmarchicus[goodtime] .- 0. * mean(calanus_finmarchicus[goodtime]), (L, L), epsilon2);
                f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                    calanus_helgolandicus[goodtime] .- 0. * mean(calanus_helgolandicus[goodtime]), (L, L), epsilon2);

                # Error field
                cpme_fin = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                    calanus_finmarchicus[goodtime], L, epsilon2);
                cpme_hel = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                    calanus_helgolandicus[goodtime], L, epsilon2);

                if makeplot
                    figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy)$(mm).jpg")         
                    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname, mainproj)

                    figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy)$(mm).jpg")
                    #make_plot_analysis(longrid, latgrid, f_helgolandicus .+ mean(calanus_helgolandicus[goodtime]), 
                    #    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname)
                end

                Dataset(resfilemonth1, "a") do ds
                    ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                    ds["error"][:,:,iii] = cpme_fin
                    ds["abundance"][:,:,iii] = f_finmarchicus
                end

                Dataset(resfilemonth2, "a") do ds
                    ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                    ds["error"][:,:,iii] = cpme_hel
                    ds["abundance"][:,:,iii] = f_helgolandicus
                end

            else
                @warn("No data to perform analysis")
            end

        end

    end
end

[ Info: Found 141 dates points for year 1959 | month 01
[ Info: Found 144 dates points for year 1959 | month 02
[ Info: Found 247 dates points for year 1959 | month 03
[ Info: Found 124 dates points for year 1959 | month 04
[ Info: Found 202 dates points for year 1959 | month 05
[ Info: Found 227 dates points for year 1959 | month 06
[ Info: Found 216 dates points for year 1959 | month 07
[ Info: Found 220 dates points for year 1959 | month 08
[ Info: Found 239 dates points for year 1959 | month 09
[ Info: Found 270 dates points for year 1959 | month 10
[ Info: Found 183 dates points for year 1959 | month 11
[ Info: Found 151 dates points for year 1959 | month 12
[ Info: Found 160 dates points for year 1960 | month 01
[ Info: Found 186 dates points for year 1960 | month 02
[ Info: Found 197 dates points for year 1960 | month 03
[ Info: Found 196 dates points for year 1960 | month 04
[ Info: Found 245 dates points for year 1960 | month 05
[ Info: Found 179 dates points for year 1960 | m

[ Info: Found 166 dates points for year 1966 | month 05
[ Info: Found 160 dates points for year 1966 | month 06
[ Info: Found 291 dates points for year 1966 | month 07
[ Info: Found 190 dates points for year 1966 | month 08
[ Info: Found 155 dates points for year 1966 | month 09
[ Info: Found 263 dates points for year 1966 | month 10
[ Info: Found 181 dates points for year 1966 | month 11
[ Info: Found 159 dates points for year 1966 | month 12
[ Info: Found 236 dates points for year 1967 | month 01
[ Info: Found 240 dates points for year 1967 | month 02
[ Info: Found 227 dates points for year 1967 | month 03
[ Info: Found 186 dates points for year 1967 | month 04
[ Info: Found 250 dates points for year 1967 | month 05
[ Info: Found 199 dates points for year 1967 | month 06
[ Info: Found 256 dates points for year 1967 | month 07
[ Info: Found 229 dates points for year 1967 | month 08
[ Info: Found 206 dates points for year 1967 | month 09
[ Info: Found 239 dates points for year 1967 | m

[ Info: Found 161 dates points for year 1973 | month 11
[ Info: Found 187 dates points for year 1973 | month 12
[ Info: Found 220 dates points for year 1974 | month 01
[ Info: Found 223 dates points for year 1974 | month 02
[ Info: Found 303 dates points for year 1974 | month 03
[ Info: Found 318 dates points for year 1974 | month 04
[ Info: Found 170 dates points for year 1974 | month 05
[ Info: Found 252 dates points for year 1974 | month 06
[ Info: Found 170 dates points for year 1974 | month 07
[ Info: Found 243 dates points for year 1974 | month 08
[ Info: Found 182 dates points for year 1974 | month 09
[ Info: Found 179 dates points for year 1974 | month 10
[ Info: Found 193 dates points for year 1974 | month 11
[ Info: Found 147 dates points for year 1974 | month 12
[ Info: Found 129 dates points for year 1975 | month 01
[ Info: Found 178 dates points for year 1975 | month 02
[ Info: Found 259 dates points for year 1975 | month 03
[ Info: Found 261 dates points for year 1975 | m

[ Info: Found 194 dates points for year 1981 | month 07
[ Info: Found 99 dates points for year 1981 | month 08
[ Info: Found 154 dates points for year 1981 | month 09
[ Info: Found 206 dates points for year 1981 | month 10
[ Info: Found 203 dates points for year 1981 | month 11
[ Info: Found 158 dates points for year 1981 | month 12
[ Info: Found 124 dates points for year 1982 | month 01
[ Info: Found 145 dates points for year 1982 | month 02
[ Info: Found 221 dates points for year 1982 | month 03
[ Info: Found 187 dates points for year 1982 | month 04
[ Info: Found 158 dates points for year 1982 | month 05
[ Info: Found 195 dates points for year 1982 | month 06
[ Info: Found 163 dates points for year 1982 | month 07
[ Info: Found 235 dates points for year 1982 | month 08
[ Info: Found 209 dates points for year 1982 | month 09
[ Info: Found 151 dates points for year 1982 | month 10
[ Info: Found 228 dates points for year 1982 | month 11
[ Info: Found 176 dates points for year 1982 | mo

[ Info: Found 170 dates points for year 1989 | month 01
[ Info: Found 142 dates points for year 1989 | month 02
[ Info: Found 150 dates points for year 1989 | month 03
[ Info: Found 160 dates points for year 1989 | month 04
[ Info: Found 161 dates points for year 1989 | month 05
[ Info: Found 194 dates points for year 1989 | month 06
[ Info: Found 133 dates points for year 1989 | month 07
[ Info: Found 146 dates points for year 1989 | month 08
[ Info: Found 182 dates points for year 1989 | month 09
[ Info: Found 208 dates points for year 1989 | month 10
[ Info: Found 202 dates points for year 1989 | month 11
[ Info: Found 151 dates points for year 1989 | month 12
[ Info: Found 162 dates points for year 1990 | month 01
[ Info: Found 184 dates points for year 1990 | month 02
[ Info: Found 131 dates points for year 1990 | month 03
[ Info: Found 147 dates points for year 1990 | month 04
[ Info: Found 201 dates points for year 1990 | month 05
[ Info: Found 164 dates points for year 1990 | m

[ Info: Found 85 dates points for year 1996 | month 07
[ Info: Found 137 dates points for year 1996 | month 08
[ Info: Found 150 dates points for year 1996 | month 09
[ Info: Found 191 dates points for year 1996 | month 10
[ Info: Found 201 dates points for year 1996 | month 11
[ Info: Found 162 dates points for year 1996 | month 12
[ Info: Found 199 dates points for year 1997 | month 01
[ Info: Found 177 dates points for year 1997 | month 02
[ Info: Found 201 dates points for year 1997 | month 03
[ Info: Found 225 dates points for year 1997 | month 04
[ Info: Found 233 dates points for year 1997 | month 05
[ Info: Found 239 dates points for year 1997 | month 06
[ Info: Found 203 dates points for year 1997 | month 07
[ Info: Found 158 dates points for year 1997 | month 08
[ Info: Found 196 dates points for year 1997 | month 09
[ Info: Found 242 dates points for year 1997 | month 10
[ Info: Found 202 dates points for year 1997 | month 11
[ Info: Found 238 dates points for year 1997 | mo

[ Info: Found 190 dates points for year 2004 | month 03
[ Info: Found 176 dates points for year 2004 | month 04
[ Info: Found 171 dates points for year 2004 | month 05
[ Info: Found 269 dates points for year 2004 | month 06
[ Info: Found 248 dates points for year 2004 | month 07
[ Info: Found 264 dates points for year 2004 | month 08
[ Info: Found 214 dates points for year 2004 | month 09
[ Info: Found 242 dates points for year 2004 | month 10
[ Info: Found 235 dates points for year 2004 | month 11
[ Info: Found 197 dates points for year 2004 | month 12
[ Info: Found 173 dates points for year 2005 | month 01
[ Info: Found 215 dates points for year 2005 | month 02
[ Info: Found 282 dates points for year 2005 | month 03
[ Info: Found 242 dates points for year 2005 | month 04
[ Info: Found 257 dates points for year 2005 | month 05
[ Info: Found 212 dates points for year 2005 | month 06
[ Info: Found 318 dates points for year 2005 | month 07
[ Info: Found 229 dates points for year 2005 | m

[ Info: Found 220 dates points for year 2011 | month 09
[ Info: Found 237 dates points for year 2011 | month 10
[ Info: Found 208 dates points for year 2011 | month 11
[ Info: Found 203 dates points for year 2011 | month 12
[ Info: Found 178 dates points for year 2012 | month 01
[ Info: Found 229 dates points for year 2012 | month 02
[ Info: Found 234 dates points for year 2012 | month 03
[ Info: Found 212 dates points for year 2012 | month 04
[ Info: Found 155 dates points for year 2012 | month 05
[ Info: Found 239 dates points for year 2012 | month 06
[ Info: Found 274 dates points for year 2012 | month 07
[ Info: Found 159 dates points for year 2012 | month 08
[ Info: Found 235 dates points for year 2012 | month 09
[ Info: Found 188 dates points for year 2012 | month 10
[ Info: Found 219 dates points for year 2012 | month 11
[ Info: Found 195 dates points for year 2012 | month 12
[ Info: Found 213 dates points for year 2013 | month 01
[ Info: Found 215 dates points for year 2013 | m

In [16]:
@info("Results writtin in directory $(resdir)");

[ Info: Results writtin in directory ../results/2deg/


## Plots
If `cartopy` could not be installed properly in Julia, you can use the Python notebook [plot_results_map_python](plot_results_map_python.ipynb) to create the figures using the netCDF files.